In [4]:
# =============================================================================
# BLOCK 1: SETUP, IMPORTS, AND DATA LOADING
# =============================================================================
# --- Basic Setup ---
import warnings
warnings.filterwarnings('ignore')
import time
print("--- Initializing ---")

# --- Library Imports ---
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import optuna

print("Libraries imported successfully.")

# --- Helper Function for Winkler Score ---
# This is our primary evaluation metric for the competition.
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    width = upper - lower
    penalty_lower = np.where(y_true < lower, (2 / alpha) * (lower - y_true), 0)
    penalty_upper = np.where(y_true > upper, (2 / alpha) * (y_true - upper), 0)
    score = width + penalty_lower + penalty_upper
    if return_coverage:
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        return np.mean(score), coverage
    return np.mean(score)

# --- Global Constants ---
N_SPLITS = 5
RANDOM_STATE = 42
DATA_PATH = './'
N_OPTUNA_TRIALS = 30 # Increase for more exhaustive search, 30 is a strong start
COMPETITION_ALPHA = 0.1

# --- Load Raw Data ---
try:
    df_train = pd.read_csv(DATA_PATH + 'dataset.csv', index_col="id", parse_dates=["sale_date"])
    df_test = pd.read_csv(DATA_PATH + 'test.csv', index_col="id", parse_dates=["sale_date"])
    print("Raw data loaded successfully.")
except FileNotFoundError:
    print("ERROR: Could not find 'dataset.csv' or 'test.csv'.")
    exit()

# --- Prepare Target Variable ---
# We store the true dollar value for final evaluation
y_true = df_train['sale_price'].copy()
# We create the log-transformed version for training our models
y_log = np.log1p(y_true)

# Add log-transformed target to df_train for feature engineering
df_train['sale_price_log'] = y_log

print("Setup complete. Raw data and target variables are ready.")

--- Initializing ---
Libraries imported successfully.
Raw data loaded successfully.
Setup complete. Raw data and target variables are ready.


In [5]:
# =============================================================================
# BLOCK 2: ULTIMATE FEATURE ENGINEERING
# =============================================================================
print("\n--- Starting Block 2: Ultimate Feature Engineering ---")

# Combine for consistent processing
df_train['is_train'] = 1
df_test['is_train'] = 0
all_data = pd.concat([df_train.drop('sale_price', axis=1), df_test], axis=0)

# --- A) Foundational & God-Tier Features ---
print("Creating foundational and contextual features...")
all_data['sale_year'] = all_data['sale_date'].dt.year
all_data['age_at_sale'] = all_data['sale_year'] - all_data['year_built']
all_data['total_bathrooms'] = all_data['bath_full'] + 0.5 * all_data['bath_half'] + 0.75 * all_data['bath_3qtr']
all_data['total_sqft'] = all_data['sqft'] + all_data['sqft_fbsmt']
all_data['was_renovated'] = (all_data['year_reno'] > 0).astype(int)
all_data['land_val_per_sqft_lot'] = all_data['land_val'] / (all_data['sqft_lot'] + 1e-6)
all_data['lot_utilization'] = all_data['total_sqft'] / (all_data['sqft_lot'] + 1e-6)

kmeans = KMeans(n_clusters=40, random_state=RANDOM_STATE, n_init='auto')
all_data['location_cluster'] = kmeans.fit_predict(all_data[['latitude', 'longitude']])

train_copy_for_aggs = all_data[all_data['is_train'] == 1].copy()
group_cols_to_agg = ['location_cluster', 'city', 'submarket']

for group_col in group_cols_to_agg:
    aggs = {'grade': ['mean', 'std'], 'age_at_sale': ['mean', 'std'], 'sale_price_log': ['mean']}
    group_aggs = train_copy_for_aggs.groupby(group_col).agg(aggs)
    group_aggs.columns = [f'{c[0]}_{c[1]}_{group_col}' for c in group_aggs.columns]
    all_data = all_data.merge(group_aggs, on=group_col, how='left')
    all_data[f'grade_vs_mean_{group_col}'] = all_data['grade'] - all_data[f'grade_mean_{group_col}']

# --- B) TF-IDF Text Features ---
print("Creating TF-IDF features for high-cardinality text...")
text_cols = ['subdivision', 'zoning']
for col in text_cols:
    all_data[col] = all_data[col].fillna('missing').astype(str)
    tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2, 4), max_features=32, binary=True)
    tfidf_matrix = tfidf.fit_transform(all_data[col])
    svd = TruncatedSVD(n_components=8, random_state=RANDOM_STATE)
    tfidf_svd = svd.fit_transform(tfidf_matrix)
    tfidf_df = pd.DataFrame(tfidf_svd, columns=[f'{col}_tfidf_svd_{i}' for i in range(8)])
    all_data = pd.concat([all_data, tfidf_df], axis=1)

# --- C) Final Cleanup ---
print("Finalizing feature set...")
cols_to_drop = ['sale_date', 'year_built', 'year_reno', 'bath_full', 'bath_half', 'bath_3qtr', 'sqft', 'sqft_fbsmt', 'latitude', 'longitude', 'subdivision', 'zoning']
all_data = all_data.drop(columns=cols_to_drop)

# Label Encoding for remaining categoricals
for col in all_data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    all_data[col] = le.fit_transform(all_data[col].astype(str))

all_data.fillna(0, inplace=True)

# Separate final datasets
X = all_data[all_data['is_train'] == 1].drop(columns=['is_train', 'sale_price_log'])
X_test = all_data[all_data['is_train'] == 0].drop(columns=['is_train', 'sale_price_log'])
X_test = X_test[X.columns]

print(f"Ultimate feature engineering complete. Total features: {X.shape[1]}")
del all_data, train_copy_for_aggs
gc.collect()


--- Starting Block 2: Ultimate Feature Engineering ---
Creating foundational and contextual features...
Creating TF-IDF features for high-cardinality text...
Finalizing feature set...
Ultimate feature engineering complete. Total features: 75


662

In [7]:
# =============================================================================
# BLOCK 3: TUNE THE MEAN PREDICTION MODEL
# =============================================================================
from sklearn.metrics import mean_squared_error # <--- IMPORT THE MISSING FUNCTION
print("\n--- STAGE 1, PART 1: Tuning Mean Prediction Model ---")

def objective_mean(trial):
    """Optuna objective function to find the best hyperparameters for the mean model."""
    train_x, val_x, train_y, val_y = train_test_split(X, y_log, test_size=0.2, random_state=RANDOM_STATE)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dval = xgb.DMatrix(val_x, label=val_y)
    
    params = {
        'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist',
        'eta': trial.suggest_float('eta', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 5, 12),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
    }
    
    model = xgb.train(params, dtrain, num_boost_round=1500,
                      evals=[(dval, 'eval')], early_stopping_rounds=50, verbose_eval=False)
    
    preds = model.predict(dval, iteration_range=(0, model.best_iteration))
    return np.sqrt(mean_squared_error(val_y, preds))

study_mean = optuna.create_study(direction='minimize')
study_mean.optimize(objective_mean, n_trials=N_OPTUNA_TRIALS)
best_params_mean = study_mean.best_params

print(f"\nMean Model Tuning Complete. Best Validation RMSE: {study_mean.best_value:.4f}")
print(f"Best params for mean model: {best_params_mean}")

[I 2025-07-05 16:49:05,301] A new study created in memory with name: no-name-cde07028-ea78-404d-8be9-a0ce6331d3b4



--- STAGE 1, PART 1: Tuning Mean Prediction Model ---


[I 2025-07-05 16:49:35,768] Trial 0 finished with value: 0.13759402577494928 and parameters: {'eta': 0.024720483553243557, 'max_depth': 8, 'subsample': 0.7602914350498581, 'colsample_bytree': 0.7362487413887749, 'min_child_weight': 1, 'lambda': 0.0001216863746901541, 'alpha': 0.2994095376993054}. Best is trial 0 with value: 0.13759402577494928.
[I 2025-07-05 16:49:54,495] Trial 1 finished with value: 0.13794838206788593 and parameters: {'eta': 0.019458768678795348, 'max_depth': 9, 'subsample': 0.6455825008512953, 'colsample_bytree': 0.7984418603635689, 'min_child_weight': 13, 'lambda': 5.6533574088665524e-06, 'alpha': 3.329362864088068e-06}. Best is trial 0 with value: 0.13759402577494928.
[I 2025-07-05 16:50:04,428] Trial 2 finished with value: 0.1376379610132431 and parameters: {'eta': 0.061120046867275075, 'max_depth': 7, 'subsample': 0.9279446047883021, 'colsample_bytree': 0.6467476867757751, 'min_child_weight': 11, 'lambda': 0.9816210563422542, 'alpha': 0.0033889079412873675}. Bes


Mean Model Tuning Complete. Best Validation RMSE: 0.1375
Best params for mean model: {'eta': 0.05966190307097686, 'max_depth': 7, 'subsample': 0.9978850431328774, 'colsample_bytree': 0.6744474983037334, 'min_child_weight': 16, 'lambda': 0.0039087357905429245, 'alpha': 0.5753645715201999}


In [8]:
# =============================================================================
# BLOCK 4: K-FOLD TRAINING OF MEAN MODEL
# =============================================================================
print("\n--- STAGE 1, PART 2: K-Fold Training of Mean Model ---")

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
oof_mean_preds = np.zeros(len(X))
test_mean_preds = np.zeros(len(X_test))
final_params_mean = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', **best_params_mean}
grade_for_stratify = pd.read_csv(DATA_PATH + 'dataset.csv')['grade']

for fold, (train_idx, val_idx) in enumerate(skf.split(X, grade_for_stratify)):
    print(f"  Mean Model - Fold {fold+1}/{N_SPLITS}...")
    dtrain = xgb.DMatrix(X.iloc[train_idx], label=y_log.iloc[train_idx])
    dval = xgb.DMatrix(X.iloc[val_idx], label=y_log.iloc[val_idx])
    model = xgb.train(final_params_mean, dtrain, num_boost_round=2000,
                      evals=[(dval, 'eval')], early_stopping_rounds=100, verbose_eval=False)
    
    oof_mean_preds[val_idx] = model.predict(dval, iteration_range=(0, model.best_iteration))
    test_mean_preds += model.predict(xgb.DMatrix(X_test), iteration_range=(0, model.best_iteration)) / N_SPLITS

print("Mean model K-Fold training complete.")


--- STAGE 1, PART 2: K-Fold Training of Mean Model ---
  Mean Model - Fold 1/5...
  Mean Model - Fold 2/5...
  Mean Model - Fold 3/5...
  Mean Model - Fold 4/5...
  Mean Model - Fold 5/5...
Mean model K-Fold training complete.


In [9]:
# =============================================================================
# BLOCK 5: TUNE THE ERROR PREDICTION MODEL
# =============================================================================
print("\n--- STAGE 2, PART 1: Tuning Error (STD) Prediction Model ---")

# Define the new target: the absolute error of the mean model's OOF predictions
error_target = np.abs(y_log - oof_mean_preds)
# We add the OOF mean predictions as a new feature for the error model
X_for_error = X.copy()
X_for_error['mean_pred_oof'] = oof_mean_preds

def objective_error(trial):
    """Optuna objective function for the error model."""
    train_x, val_x, train_y, val_y = train_test_split(X_for_error, error_target, test_size=0.2, random_state=RANDOM_STATE)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dval = xgb.DMatrix(val_x, label=val_y)
    params = {
        'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist',
        'eta': trial.suggest_float('eta', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 9),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 25),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
    }
    model = xgb.train(params, dtrain, num_boost_round=1500,
                      evals=[(dval, 'eval')], early_stopping_rounds=50, verbose_eval=False)
    preds = model.predict(dval, iteration_range=(0, model.best_iteration))
    return np.sqrt(mean_squared_error(val_y, preds))

study_error = optuna.create_study(direction='minimize')
study_error.optimize(objective_error, n_trials=N_OPTUNA_TRIALS)
best_params_error = study_error.best_params

print(f"\nError Model Tuning Complete. Best Validation RMSE: {study_error.best_value:.4f}")
print(f"Best params for error model: {best_params_error}")

[I 2025-07-05 16:59:46,482] A new study created in memory with name: no-name-914e4312-6360-4cce-9cb4-591698f9b3d7



--- STAGE 2, PART 1: Tuning Error (STD) Prediction Model ---


[I 2025-07-05 16:59:48,553] Trial 0 finished with value: 0.08921814228036218 and parameters: {'eta': 0.06808667628017309, 'max_depth': 9, 'subsample': 0.9552038469337909, 'colsample_bytree': 0.8985898959633798, 'min_child_weight': 16, 'lambda': 1.9554977369966e-05}. Best is trial 0 with value: 0.08921814228036218.
[I 2025-07-05 16:59:54,238] Trial 1 finished with value: 0.08902112588874593 and parameters: {'eta': 0.020849001534289098, 'max_depth': 9, 'subsample': 0.8879254596158186, 'colsample_bytree': 0.6218093858138868, 'min_child_weight': 22, 'lambda': 0.00023526708744078963}. Best is trial 1 with value: 0.08902112588874593.
[I 2025-07-05 16:59:57,048] Trial 2 finished with value: 0.08935693425682914 and parameters: {'eta': 0.02804821578145234, 'max_depth': 7, 'subsample': 0.6175696151789862, 'colsample_bytree': 0.6914934319112296, 'min_child_weight': 15, 'lambda': 0.0038294116881693687}. Best is trial 1 with value: 0.08902112588874593.
[I 2025-07-05 16:59:59,499] Trial 3 finished w


Error Model Tuning Complete. Best Validation RMSE: 0.0889
Best params for error model: {'eta': 0.01015003482647279, 'max_depth': 8, 'subsample': 0.9161208842907641, 'colsample_bytree': 0.6664376176064557, 'min_child_weight': 8, 'lambda': 0.001107878407698638}


In [10]:
# =============================================================================
# BLOCK 6: K-FOLD TRAINING OF ERROR MODEL
# =============================================================================
print("\n--- STAGE 2, PART 2: K-Fold Training of Error Model ---")
# Add mean predictions as a feature to the test set as well
X_test_for_error = X_test.copy()
X_test_for_error['mean_pred_oof'] = test_mean_preds

oof_error_preds = np.zeros(len(X))
test_error_preds = np.zeros(len(X_test))
final_params_error = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', **best_params_error}

for fold, (train_idx, val_idx) in enumerate(skf.split(X_for_error, grade_for_stratify)):
    print(f"  Error Model - Fold {fold+1}/{N_SPLITS}...")
    dtrain = xgb.DMatrix(X_for_error.iloc[train_idx], label=error_target.iloc[train_idx])
    dval = xgb.DMatrix(X_for_error.iloc[val_idx], label=error_target.iloc[val_idx])
    
    model = xgb.train(final_params_error, dtrain, num_boost_round=2000,
                      evals=[(dval, 'eval')], early_stopping_rounds=100, verbose_eval=False)
    
    oof_error_preds[val_idx] = model.predict(dval, iteration_range=(0, model.best_iteration))
    test_error_preds += model.predict(xgb.DMatrix(X_test_for_error), iteration_range=(0, model.best_iteration)) / N_SPLITS

print("Error model K-Fold training complete.")


--- STAGE 2, PART 2: K-Fold Training of Error Model ---
  Error Model - Fold 1/5...
  Error Model - Fold 2/5...
  Error Model - Fold 3/5...
  Error Model - Fold 4/5...
  Error Model - Fold 5/5...
Error model K-Fold training complete.


In [14]:
# =============================================================================
# FINAL BLOCK (CORRECTED): ASYMMETRIC CALIBRATION IN LOG SPACE
# =============================================================================
print("\n--- Starting Final Calibration and Submission (Corrected) ---")

# --- 1. Asymmetric Grid Search for Best Multipliers (in Log Space) ---
# The oof_mean_preds and oof_error_preds are already in log scale, which is what we need.
# The y_true is the original dollar value, which is correct for the winkler_score function.

print("Searching for best asymmetric multipliers (a, b) in log space...")
best_a, best_b, best_metric = 2.0, 2.0, float('inf')

# Let's search a wider range for the multipliers now.
for a in np.arange(1.0, 3.51, 0.05):
    for b in np.arange(1.0, 3.51, 0.05):
        
        # --- CONSTRUCT THE INTERVAL IN LOG SPACE ---
        # The error predictions must be positive
        error_pred_log = np.clip(oof_error_preds, 0, None) 
        
        # Calculate lower and upper bounds in log scale
        lower_log = oof_mean_preds - error_pred_log * a
        upper_log = oof_mean_preds + error_pred_log * b
        
        # --- CONVERT BOUNDS TO DOLLAR SCALE FOR EVALUATION ---
        lower_dollar = np.expm1(lower_log)
        upper_dollar = np.expm1(upper_log)
        
        # Safety check: ensure upper > lower
        upper_dollar = np.maximum(lower_dollar, upper_dollar)

        metric, coverage = winkler_score(y_true, lower_dollar, upper_dollar, alpha=0.1, return_coverage=True)
        
        # We only care about solutions that have reasonable coverage
        if coverage > 0.80 and metric < best_metric:
            best_metric = metric
            best_a = a
            best_b = b
            print(f"New Best! a={best_a:.2f}, b={best_b:.2f}, Score={best_metric:,.2f}, Cov={coverage:.2%}")

print(f"\nGrid search complete. Final OOF Score: {best_metric:,.2f}. Best multipliers: a={best_a:.2f}, b={best_b:.2f}")

# --- 2. Create Final Submission ---
print("\nCreating final submission file...")

# Use the same logic on the test set predictions
error_pred_test_log = np.clip(test_error_preds, 0, None)
lower_final_log = test_mean_preds - error_pred_test_log * best_a
upper_final_log = test_mean_preds + error_pred_test_log * best_b

# Convert final bounds to dollar scale
final_lower = np.expm1(lower_final_log)
final_upper = np.expm1(upper_final_log)

# Final safety checks
final_upper = np.maximum(final_lower, final_upper)
final_lower = np.clip(final_lower, y_true.min(), None) # Clip at min training price
final_upper = np.clip(final_upper, y_true.min(), None)


submission_df = pd.DataFrame({
    'id': X_test.index,
    'pi_lower': final_lower,
    'pi_upper': final_upper
})

submission_df.to_csv('submission_mean-error_final_v5.csv', index=False)
print("\n'submission_mean-error_final_v5.csv' created successfully!")
display(submission_df.head())


--- Starting Final Calibration and Submission (Corrected) ---
Searching for best asymmetric multipliers (a, b) in log space...
New Best! a=1.05, b=2.95, Score=405,057.86, Cov=80.05%
New Best! a=1.10, b=2.55, Score=376,556.89, Cov=80.03%
New Best! a=1.15, b=2.35, Score=362,970.16, Cov=80.17%
New Best! a=1.20, b=2.20, Score=353,820.41, Cov=80.29%
New Best! a=1.25, b=2.05, Score=347,248.47, Cov=80.17%
New Best! a=1.30, b=1.95, Score=343,016.43, Cov=80.24%
New Best! a=1.30, b=2.00, Score=342,649.79, Cov=80.62%
New Best! a=1.30, b=2.05, Score=342,579.70, Cov=81.01%
New Best! a=1.35, b=1.85, Score=340,403.06, Cov=80.10%
New Best! a=1.35, b=1.90, Score=339,430.51, Cov=80.59%
New Best! a=1.35, b=1.95, Score=338,776.20, Cov=81.05%
New Best! a=1.35, b=2.00, Score=338,409.55, Cov=81.43%
New Best! a=1.35, b=2.05, Score=338,339.47, Cov=81.83%
New Best! a=1.40, b=1.80, Score=337,889.53, Cov=80.37%
New Best! a=1.40, b=1.85, Score=336,574.82, Cov=80.89%
New Best! a=1.40, b=1.90, Score=335,602.27, Cov

,id,pi_lower,pi_upper
0,200000,802643.333125,1.120053e+06
1,200001,602250.191158,8.518079e+05
2,200002,416796.373025,6.263170e+05
3,200003,302598.928411,4.378161e+05
4,200004,349629.046804,8.168408e+05
